## Sommaire :

* [**1.Problem setup**](#0)
    * [1.1. BS market model](#1_1)
    * [1.2. American payoff](#1_2)
    * [1.3. Snell enveloppe and supermartingale structure](#1_3)

* [**2.Regression method : Longstaff Schwartz**](#1)
    * [2.1. BS dynamics](#1_1)
    * [2.2. Backward Induction: Longstaff–Schwartz ](#1_2)

* [**3...**](#2)
    * [3.1. ...](#2_1)
    * [3.2. ...](#2_2)
    * [3.3. ...](#2_3)


<a id='0'></a>
# 1. Problem setup :

We work on a filtered probability space  
$(\Omega,\mathcal{F}, (\mathcal{F}_t)_{0\le t\le T},\mathbb{Q})$  
satisfying the usual conditions.  
All pricing will be conducted under the **risk-neutral measure** $\mathbb{Q}$.

---

## 1.1 Black–Scholes Market Model

We consider a single risky asset $S$ whose dynamics under $\mathbb{Q}$ is:

$$
dS_t = r S_t\,dt + \sigma S_t\, dW_t,
$$

where  
- $r$ is the constant short rate,  
- $\sigma>0$ the constant volatility,  
- $(W_t)$ a standard Brownian motion under $\mathbb{Q}$.

The **discount factor** is:

$$
D(t,T) = e^{-r(T-t)}.
$$

The **discounted stock price**  
$\tilde S_t = D(0,t) S_t$  
satisfies:

$$
d\tilde S_t = \sigma \tilde S_t\, dW_t,
$$

so $(\tilde S_t)$ is a **$\mathbb{Q}$-martingale**.  
This property is what allows Monte Carlo pricing.

---

## 1.2 American Put Payoff

We study the American **put** with strike $K$ and maturity $T$.  
If the holder exercises at time $t$, the payoff is:

$$
G_t = (K - S_t)_+.
$$

We define its **discounted payoff process**:

$$
\tilde G_t = \beta_t G_t = e^{-rt}(K - S_t)_+.
$$

The American option price is:

$$
P_0 = \sup_{\tau \in \mathcal{T}} 
\mathbb{E}^{\mathbb{Q}}[\tilde G_\tau],
$$

where $\mathcal{T}$ is the set of stopping times valued in $[0,T]$.

This is the **optimal stopping formulation**, common to:
- Monte Carlo backward induction (LSM, Andersen–Broadie, etc.),
- PDE variational inequalities (free boundary),
- dual formulations (Glasserman 8.7).

---

## 1.3 Snell Envelope and Supermartingale Structure

A fundamental fact (Lamberton–Lapeyre, Glasserman 8.1) is:

> The discounted value process of any American option is the **Snell envelope**  
> of the discounted payoff.

### Snell Envelope  
Define:

$$
Y_t = \operatorname*{ess\,sup}_{\tau\in\mathcal{T}_t}
\mathbb{E}^{\mathbb{Q}}[\tilde G_\tau \mid \mathcal{F}_t].
$$

Then:

- $(Y_t)$ is a **$\mathbb{Q}$-supermartingale**,  
- $Y_t \ge \tilde G_t$ for all $t$,  
- $Y$ is the *smallest* supermartingale dominating $\tilde G$.

A process $(Y_t)$ is a **supermartingale** if:

$$
\mathbb{E}[Y_t \mid \mathcal{F}_s] \le Y_s
\quad (s \le t).
$$

Intuition:
- **Discounted continuation values cannot increase in expectation**.
- This makes early-exercise analysis compatible with dynamic programming.

### Optimal Exercise Time

Define the first hitting time:

$$
\tau^\star = \inf\{t : Y_t = \tilde G_t\}.
$$

Then $\tau^\star$ is an **optimal stopping time** and:

$$
P_0 = Y_0 = \mathbb{E}^{\mathbb{Q}}[\tilde G_{\tau^\star}].
$$

This structure is exactly what underlies:
- regression Monte Carlo (LSM): approximation of $Y_t$ from below,
- dual Monte Carlo (Andersen–Broadie): construction of upper bounds via martingales,
- PDE free boundary: region where $Y_t = \tilde G_t$.


In [21]:
# Modules 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm
import time
from math import log, sqrt, exp
from dataclasses import dataclass, field
from typing import Optional

<a id='1'></a>
# 2. Regression methods :

<a id='1_1'></a>
## 2.1 BS dynamics :


Sous $\mathbb{Q}$, la dynamique de $\mathcal{S}$ s'écrit :

$$
\frac{dS_t}{S_t} = r \, dt + \sigma \, dW_t
$$

**Solution de l'EDS :** (Pas de schéma nécessaire)

$$
d(\ln(S_t)) = \frac{1}{S_t} dS_t - \frac{1}{2} \frac{1}{S_t^2} (dS_t)^2
\implies d(\ln(S_t)) = \frac{1}{S_t} \big(S_t (r  \, dt + \sigma \, dW_t)\big) - \frac{1}{2} \sigma^2 dt
\implies d(\ln(S_t)) = \left(r  - \frac{1}{2} \sigma^2\right) dt + \sigma dW_t
$$

$$
\ln(S_t) = \ln(S_0) + \left(r  - \frac{1}{2} \sigma^2\right)t + \sigma W_t
\implies S_t = S_0 \exp\left(\left(r  - \frac{1}{2} \sigma^2\right)t + \sigma W_t\right)
$$


$$
\implies S_{t_k} = S_{t_{k-1}} \exp\left(\left(r  - \frac{1}{2} \sigma^2\right)(t_k - t_{k-1}) + \sigma \left(W_{t_k} - W_{t_{k-1}}\right)\right)
$$



$$
\implies
\boxed{ \ln \Big(\tfrac{S_{t_k}}{S_{t_{k-1}}}\Big) \,\big|\, \mathcal{F}_s 
\sim \mathcal{N}\!\Big( \big(r-\tfrac12\sigma^2\big)(t_k - t_{k-1}),\; \sigma^2(t_k - t_{k-1}) \Big) }
$$

---

In [22]:
# =============================================================================
#                         BLACK–SCHOLES MODEL
# =============================================================================

@dataclass
class BlackScholes:
    """
    Black–Scholes model under the risk-neutral measure 𝒬.

    Parameters
    ----------
    S0 : float
        Initial underlying price S(0).
    r : float
        Constant risk-free rate (annual, continuously compounded).
    sigma : float
        Constant volatility of the underlying (annual).
    q : float
        Constant dividend (annual, continuously compounded).

    Notes
    -----
    Under 𝒬, the asset follows the geometric Brownian motion (GBM):
        dS_t = (r-q) S_t dt + σ S_t dW_t

    The closed-form solution used for simulation is:
        S_{t+dt} = S_t * exp( ((r-q) - 0.5 σ²) dt + σ sqrt(dt) Z ),
    where Z ~ N(0,1).
    """

    S0: float
    r: float
    sigma: float
    q: float = 0.0  # Value by default

    # -------------------------------------------------------------------------
    def simulate_paths(
        self,
        T: float,
        n_steps: int,
        n_paths: int,
        seed: Optional[int] = None
    ) -> np.ndarray:
        """
        Simulate GBM paths {S_t} under the risk-neutral measure 𝒬.

        Parameters
        ----------
        T : float
            Maturity horizon in years.
        n_steps : int
            Number of time discretization steps on [0, T].
        n_paths : int
            Number of Monte Carlo trajectories to simulate.
        seed : int, optional
            Seed for the random number generator.

        Returns
        -------
        np.ndarray of shape (n_paths, n_steps + 1)
            Simulated price paths. Column k corresponds to time t_k = k*T/n_steps.

        Method
        ------
        Uses the exact discretization of GBM:
            S_{t+dt} = S_t * exp( ((r-q) - 0.5 σ²) dt + σ sqrt(dt) Z )

        Steps
        -----
        1. Initialize matrix of paths.
        2. For each time step, draw Z ~ N(0,1).
        3. Update prices using the GBM exponential formula.
        """

        rng = np.random.default_rng(seed)
        dt   = T / n_steps
        drift = ((self.r - self.q) - 0.5 * self.sigma**2) * dt
        vol   = self.sigma * np.sqrt(dt)

        paths = np.zeros((n_paths, n_steps + 1))
        paths[:, 0] = self.S0

        for t in range(1, n_steps + 1):
            z = rng.standard_normal(n_paths)
            paths[:, t] = paths[:, t - 1] * np.exp(drift + vol * z)

        return paths

<a id='1_2'></a>
## 2.2 Backward Induction: Longstaff–Schwartz 

**Problem formulation**

We know that the (risk–neutral) price of the American claim can be written as
$$
P_0 \;=\; \sup_{\tau \in \mathcal T} 
\mathbb{E}^{\mathbb{Q}}\big[\tilde G_\tau\big],
$$
where $\mathcal T$ is the set of stopping times taking values in the exercise dates
$\{t_0,\dots,t_M\}$, and $\tilde G_t$ is the discounted payoff process.

---

**Dynamic programming formulation**

On the discrete exercise grid $\{t_0,\dots,t_M\}$ we define the value function
$$
\hat P_m(x) \;=\; \tilde G_m(x), 
\qquad m = M,
$$
and for $i = M, M-1, \dots, 1$ we use the backward induction (Bellman) recursion
$$
\hat P_{i-1}(x) \;=\; 
\max\Big(
G_{i-1}(x),
\;\mathbb{E}^{\mathbb{Q}}\!\Big[
D(t_{i-1},t_i)\,\hat P_i(S_{t_i})
\;\Big|\; S_{t_{i-1}} = x
\Big]
\Big).
$$

In words:

- $G_{i-1}(x)$ is the *immediate exercise value* at time $t_{i-1}$ if the state is $S_{t_{i-1}}=x$.
- The conditional expectation term is the *continuation value*: the discounted expected value of keeping the option alive until (at least) the next exercise date $t_i$.

Finally, the initial price is obtained as
$$
P_0 = \hat P_0(S_{t_0}).
$$

In the Longstaff–Schwartz algorithm, the key idea is to **approximate**
the conditional expectation
$$
\mathbb{E}^{\mathbb{Q}}\!\Big[
D(t_{i-1},t_i)\,\hat P_i(S_{t_i})
\;\Big|\; S_{t_{i-1}} 
\Big]
$$
by a regression on a set of basis functions of $S_{t_{i-1}}$, estimated from Monte Carlo paths.

---

## From Dynamic Programming (DPP) to Stopping Times

Before introducing the parametrization by $\theta$, it is useful to see how **stopping times naturally arise** from the **Dynamic Programming Principle (DPP)**.

Consider discrete exercise dates
$$
t_0 < t_1 < \dots < t_M,
$$
and a Markov state process $(S_{t_i})_{i=0,\dots,M}$.  
Define the value function:
$$
V_i(x) = \text{value of the American claim at time } t_i
\text{ given } S_{t_i} = x.
$$

The DPP states:
$$
V_i(x)
= \max\Big(
G_i(x),\;
\mathbb{E}\big[ e^{-r\Delta t}\, V_{i+1}(S_{t_{i+1}}) 
\mid S_{t_i} = x\big]
\Big),
$$
where:
- $G_i(x)$ is the **immediate exercise payoff** at time $t_i$,
- the conditional expectation is the **continuation value**.

This gives a **local decision rule** at each time $t_i$:

- if
  $$
  G_i(S_{t_i}) \;\ge\; 
  \mathbb{E}\big[ e^{-r\Delta t}\, V_{i+1}(S_{t_{i+1}}) 
  \mid S_{t_i}\big],
  $$
  then it is optimal to **exercise** at $t_i$;

- otherwise, it is optimal to **continue**.

By applying this rule iteratively as time evolves, we obtain a **random exercise time**:
$$
\tau^\star
= \inf\{ t_i : G_i(S_{t_i}) = V_i(S_{t_i}) \},
$$
i.e. the **first exercise date at which immediate exercise is optimal**.

Because the decision at $t_i$ only uses information available at time $t_i$,  
$\tau^\star$ is a **stopping time**.  

This leads to the equivalent formulation:
$$
V_0 = \sup_{\tau \in \mathcal T} 
\mathbb{E}\big[ G_\tau(S_\tau) \big],
$$
where $\mathcal T$ is the set of stopping times taking values in $\{t_0,\dots,t_M\}$.

- DPP gives a **backward recursion** to compute $V_i(x)$.
- Stopping times give a **probabilistic description** of the optimal exercise strategy.

The parametrization with $\theta$ (below) simply restricts the set of stopping times to a **tractable family** $\{\tau(\theta): \theta\in\Theta\}$.

---

## Interpretation of $\theta$ in Parametric Stopping Rules vs. Longstaff–Schwartz

In Glasserman’s general framework for American-style optimal stopping, one considers a **family of stopping rules**
$$
\tau(\theta), \quad \theta \in \Theta,
$$
where $\theta$ is a vector of parameters describing how the stopping rule behaves.

The corresponding approximate value is
$$
V_0^\theta \;=\; 
\sup_{\theta\in\Theta} 
\mathbb{E}\!\left[ h_{\tau(\theta)}(X_{\tau(\theta)}) \right].
$$

---


###  The Case of Longstaff–Schwartz (LSM)

For the Longstaff–Schwartz algorithm:

- We approximate the continuation value at time $t_i$ by a regression:
  $$
  \widehat C_i(x;\theta)
  = \sum_{k=0}^{K} \beta_{i,k} \, \phi_k(x),
  $$
  where $\phi_k$ are basis functions (e.g. $1, x, x^2, \dots$).

- The **parameters** $\theta$ correspond to **all regression coefficients**:
  $$
  \theta =
  (\beta_{1,0},\ldots,\beta_{1,K_1},\dots,
   \beta_{M-1,0},\ldots,\beta_{M-1,K_{M-1}}).
  $$

- The stopping rule is then:
  $$
  \tau(\theta) = 
  \inf\big\{ t_i : G(S_{t_i}) 
  \ge \widehat C_i(S_{t_i};\theta)\big\},
  $$
  i.e. the **first exercise time where the payoff exceeds (or equals) the approximated continuation value**.

Thus, in LSM:

- $\theta$ does **not** directly define a barrier or geometric exercise region,
- instead, $\theta$ defines the **shape of the continuation value function**,  
- and the exercise rule follows automatically from the comparison:
  $$
  \text{exercise at } t_i 
  \quad\text{if}\quad 
  G(S_{t_i}) \ge \widehat C_i(S_{t_i};\theta).
  $$

---

### Summary

- **DPP** → describes the value function via a backward recursion  
  and implicitly defines an optimal stopping time $\tau^\star$.

- **Parametric approach (Glasserman)** → restricts to a family of stopping times  
  $\{\tau(\theta)\}$, with $\theta$ describing the exercise rule (barriers, boundaries, etc.).

- **Longstaff–Schwartz** → chooses $\theta$ as regression coefficients used to approximate  
  continuation values, which in turn define an approximate stopping time  
  $\widehat\tau = \tau(\theta)$ via the rule “exercise if payoff ≥ continuation”.



In [23]:
# =============================================================================
#                    PRICING ENGINE  —  LONGSTAFF–SCHWARTZ
# =============================================================================

class PricingEngine:
    """
    Pricing engine using Monte Carlo and regression-based algorithms.

    Parameters
    ----------
    model : BlackScholes
        Instance containing the risk-neutral dynamics for S_t.

    Notes
    -----
    This class acts as a wrapper: it does not store parameters,
    but delegates simulation to the Black–Scholes model.
    """

    def __init__(self, model: BlackScholes):
        self.model = model

    # -------------------------------------------------------------------------
    def simulate_paths(
        self,
        T: float,
        n_steps: int,
        n_paths: int,
        seed: Optional[int] = None
    ) -> np.ndarray:
        """
        Wrapper for Black–Scholes simulation.
        """
        return self.model.simulate_paths(T, n_steps, n_paths, seed)

    # -------------------------------------------------------------------------
    def price_american_put_LSM(
        self,
        K: float,
        T: float,
        n_steps: int,
        n_paths: int,
        type: str,
        basis: str = "poly2",
        seed: Optional[int] = None
    ) -> float:
        """
        Price an American option using the Longstaff–Schwartz algorithm.

        Parameters
        ----------
        K : float
            Strike price of the put option.
        T : float
            Maturity in years.
        n_steps : int
            Number of discrete exercise dates.
        n_paths : int
            Number of Monte Carlo paths.
        type : {"call", "put"}
            Type of payoff 
        basis : {"poly2", "poly3"}
            Regression basis for the continuation value:
                - "poly2": [1, S, S²]
                - "poly3": [1, S, S², S³]
        seed : int, optional
            Seed for reproducibility.

        Returns
        -------
        float
            Estimated American option price.

        Method (Longstaff–Schwartz)
        ----------------------------
        Backward dynamic programming with regression:

        1. Simulate paths S(t_k).
        2. Compute intrinsic value: payoff_put = max(K - S, 0), payoff_call = max(S - K, 0).
        3. Initialize cashflows CF = payoff at maturity.
        4. For t = n_steps-1 ... 1 (backward):
            a) Identify in-the-money paths (ITM).
            b) Regress discounted future CF on basis(S_t).
            c) Estimate continuation value C_hat.
            d) Exercise if payoff > C_hat.
        5. The American put value is the mean discounted CF at t=0.

        Notes
        -----
        - LSM provides a *lower bound* of the true price.
        - For upper bounds, use Andersen–Broadie (not implemented here).
        """

        # 1. Simulate paths
        S = self.simulate_paths(T, n_steps, n_paths, seed)
        dt   = T / n_steps
        disc = np.exp(-self.model.r * dt) # Factor of actualisation for the backward induction

        # 2. Payoff matrix
        if type == "put" :
            payoff = np.maximum(K - S, 0.0)
        else :
            payoff = np.maximum(S - K, 0.0)

        # 3. Final cashflows = payoff at maturity
        CF = payoff[:, -1].copy()

        # ------------------------------------------------------------------
        # BACKWARD INDUCTION
        # ------------------------------------------------------------------
        for t in range(n_steps - 1, 0, -1):

            # Paths in-the-money at time t
            itm = payoff[:, t] > 0   # Bool vector true if ITM
            S_itm = S[itm, t]        # Is all price wich are ITM at t

            # If no ITM, just discount continuation
            if len(S_itm) == 0:
                CF *= disc
                continue

            # Discounted future cashflows (continuation values)
            Y = CF[itm] * disc

            # ----- Regression basis -----
            if basis == "poly2":
                X = np.column_stack([np.ones(len(S_itm)), S_itm, S_itm**2])
            elif basis == "poly3":
                X = np.column_stack([np.ones(len(S_itm)), S_itm, S_itm**2, S_itm**3])
            else:
                raise ValueError("Unknown regression basis.")

            # Least-squares regression : C_hat ≈ E[ CF_{t+1} | S_t ]
            beta = np.linalg.lstsq(X, Y, rcond=None)[0]
            C_hat = X @ beta

            # ----- Exercise decision -----
            exercise = payoff[itm, t] > C_hat

            # Replace continuation by exercise payoff for exercised paths
            CF[itm] = np.where(exercise, payoff[itm, t], Y)

            # Non-ITM paths simply discount continuation
            CF[~itm] *= disc

        # ------------------------------------------------------------------
        # Final: discount once more to t=0
        # ------------------------------------------------------------------
        price = disc * CF.mean()
        return float(price)

# Verif (a supprimer)

In [24]:
def european_put_bs(S0: float, K: float, r: float, sigma: float, T: float) -> float:
    """
    Closed-form Black–Scholes price of a European Put.

    P = K e^{-rT} N(-d2) - S0 N(-d1)
    
    Parameters
    ----------
    S0 : float
        Initial spot price
    K : float
        Strike
    r : float
        Constant interest rate
    sigma : float
        Volatility
    T : float
        Maturity

    Returns
    -------
    float
        Black–Scholes European Put price.
    """
    d1 = (log(S0 / K) + (r + 0.5 * sigma**2) * T) / (sigma * sqrt(T))
    d2 = d1 - sigma * sqrt(T)
    return K * exp(-r * T) * norm.cdf(-d2) - S0 * norm.cdf(-d1)



def test_pricer():
    """
    Simple functional test of the PricingEngine + LSM method.

    1) Instantiate a Black–Scholes model.
    2) Price an American Put via LSM.
    3) Price a European Put via Black–Scholes closed formula.
    4) Print & compare (American ≥ European).
    """

    # --- Parameters ---
    S0    = 100
    K     = 100
    r     = 0.05
    sigma = 0.2
    T     = 1.0

    # --- Instantiate model ---
    bs = BlackScholes(S0=S0, r=r, sigma=sigma)
    engine = PricingEngine(bs)

    # --- American Put via LSM ---
    amer = engine.price_american_put_LSM(
        K=K, T=T,
        n_steps=50,
        n_paths=20000,
        type = "put",
        basis="poly2",
        seed=123
    )

    # --- European Put (closed form) ---
    euro = european_put_bs(S0, K, r, sigma, T)

    # --- Display ---
    print("=== TEST PRICING ENGINE ===")
    print(f"European Put (BS closed form):   {euro:.6f}")
    print(f"American Put (LSM estimate):     {amer:.6f}")
    print(f"Difference (Amer - Euro):        {amer - euro:.6f}")

    # sanity check
    if amer < euro:
        print("⚠️ Warning: American < European (too few paths or regression issue).")
    else:
        print("✅ OK: American ≥ European (expected).")

    return euro, amer


In [25]:
test_pricer()

=== TEST PRICING ENGINE ===
European Put (BS closed form):   5.573526
American Put (LSM estimate):     6.045996
Difference (Amer - Euro):        0.472470
✅ OK: American ≥ European (expected).


(5.573526022256971, 6.045995958948577)

In [26]:
# =============================================================================
#            PRICING ENGINE — LONGSTAFF–SCHWARTZ (LOW + HIGH)
# =============================================================================

class PricingEngine:
    def __init__(self, model: BlackScholes):
        self.model = model

    # -------------------------------------------------------------------------
    def simulate_paths(self, T, n_steps, n_paths, seed=None):
        return self.model.simulate_paths(T, n_steps, n_paths, seed)

    # =========================================================================
    #                        LOW ESTIMATOR (LSM CLASSIQUE)
    # =========================================================================
    def price_american_put_LSM_low(
        self, K, T, n_steps, n_paths, type="put", basis="poly2", seed=None
    ):
        S = self.simulate_paths(T, n_steps, n_paths, seed)
        dt   = T / n_steps
        disc = np.exp(-self.model.r * dt)

        # Payoff
        if type == "put":
            payoff = np.maximum(K - S, 0.0)
        else:
            payoff = np.maximum(S - K, 0.0)

        # Final CF = payoff at maturity
        CF = payoff[:, -1].copy()

        # ---------------- BACKWARD LOOP (standard LSM low estimator) ----------
        for t in range(n_steps - 1, 0, -1):

            itm = payoff[:, t] > 0
            S_itm = S[itm, t]

            if len(S_itm) == 0:
                CF *= disc
                continue

            # Discount continuation
            Y = CF[itm] * disc

            # Regression
            if basis == "poly2":
                X = np.column_stack([np.ones(len(S_itm)), S_itm, S_itm**2])
            else:
                X = np.column_stack([np.ones(len(S_itm)), S_itm, S_itm**2, S_itm**3])

            beta  = np.linalg.lstsq(X, Y, rcond=None)[0]
            C_hat = X @ beta

            # EXERCISE DECISION (policy-based)
            exercise = payoff[itm, t] > C_hat

            # If exercise → replace continuation with payoff
            CF[itm] = np.where(exercise, payoff[itm, t], Y)

            # Paths not ITM just discount
            CF[~itm] *= disc

        return float(disc * CF.mean())

    # =========================================================================
    #                        HIGH ESTIMATOR
    # =========================================================================
    def price_american_put_LSM_high(
        self, K, T, n_steps, n_paths, type="put", basis="poly2", seed=None
    ):
        S = self.simulate_paths(T, n_steps, n_paths, seed)
        dt   = T / n_steps
        disc = np.exp(-self.model.r * dt)

        if type == "put":
            payoff = np.maximum(K - S, 0.0)
        else:
            payoff = np.maximum(S - K, 0.0)

        # Final value = payoff
        V = payoff[:, -1].copy()

        # --------------- BACKWARD LOOP (HIGH ESTIMATOR) -----------------------
        for t in range(n_steps - 1, 0, -1):

            itm = payoff[:, t] > 0
            S_itm = S[itm, t]

            # Discount continuation
            V *= disc

            if len(S_itm) == 0:
                continue

            # Need continuation only for ITM
            Y_future = V[itm]

            # Regression basis
            if basis == "poly2":
                X = np.column_stack([np.ones(len(S_itm)), S_itm, S_itm**2])
            else:
                X = np.column_stack([np.ones(len(S_itm)), S_itm, S_itm**2, S_itm**3])

            beta  = np.linalg.lstsq(X, Y_future, rcond=None)[0]
            C_hat = X @ beta

            # ------- KEY DIFFERENCE: MAX between payoff and estimated cont. -----
            # High estimator updates VALUE, not CF
            # Even if regression says "continue", we MAX anyway (Jensen +)
            V[itm] = np.maximum(payoff[itm, t], C_hat)

        return float(V.mean())


In [28]:
model  = BlackScholes(S0=100, r=0.05, sigma=0.2)
engine = PricingEngine(model)


low  = engine.price_american_put_LSM_low(
    K=100, T=1.0, n_steps=50, n_paths=20000, seed=42
)

high = engine.price_american_put_LSM_high(
    K=100, T=1.0, n_steps=50, n_paths=20000, seed=42
)

print("LOW estimator =", low)
print("HIGH estimator =", high)

LOW estimator = 6.084787640730125
HIGH estimator = 6.188318073333814
